In [3]:
import datetime
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
from datetime import datetime as dt
from xgboost import XGBRegressor
from math import sqrt
%run src/columns.py
%run src/data-cleaning.py
%run src/feature-engineering.py
%run src/modeling.py

In [5]:
csv_file_names = ('LoanStats3a_securev1', 'LoanStats3b_securev1', 'LoanStats3c_securev1', 'LoanStats3d_securev1',
                  'LoanStats_securev1_2016Q1', 'LoanStats_securev1_2016Q2', 'LoanStats_securev1_2016Q3',
                  'LoanStats_securev1_2016Q4', 'LoanStats_securev1_2017Q1', 'LoanStats_securev1_2017Q2', 
                  'LoanStats_securev1_2017Q3', 'LoanStats_securev1_2017Q4', 'LoanStats_securev1_2018Q1')

loans = load_data(csv_file_names, columns_to_use, number_of_rows=500)
loans = drop_loans_not_complete(loans)
loans = drop_loan_status(loans)
loans = drop_joint_applicant_loans(loans)
loans = fix_rate_cols(loans)
loans = fix_date_cols(loans)
loans = clean_loan_term_col(loans)
loans = only_include_36_month_loans(loans)
loans = clean_employment_length(loans)
loans = create_missing_data_boolean_columns(loans)
loans = fill_nas(loans, value=-99)
loans = add_issue_date_and_month(loans)
loans = add_supplemental_rate_data(loans)
loans = create_rate_difference_cols(loans)
loans = create_months_since_earliest_cl_col(loans)
loans = create_loan_life_months_col(loans)
loans['roi'] = create_roi_col(loans['total_rec_prncp'], loans['total_rec_int'], loans['loan_amnt'], loans['loan_life_months'])
loans['roi'].replace(np.inf, 0, inplace=True)
loans = change_data_types(loans)
loans = create_dummy_cols(loans)
loans = drop_unnecessary_cols(loans)
# Loans before this date will be used for training.
# Loans after this date used for testing.
cutoff_date = '2015-09-01'
training_loans, testing_loans = get_training_and_testing_data(loans, cutoff_date)
X_train, y_train = split_data_into_labels_and_target(training_loans)
X_test, y_test = split_data_into_labels_and_target(testing_loans)

In [6]:
model = XGBRegressor()
fit_model = train_model(model, X_train, y_train)

get_predictions(fit_model, X_test)[:5]

array([-16.20946  ,   3.7979133,   3.151183 ,   6.8159075,  -2.195511 ],
      dtype=float32)

Get Payment History Data

In [8]:
training_loans.head()

,id,loan_amnt,int_rate,installment,emp_length,annual_inc,issue_d,dti,delinq_2yrs,fico_range_low,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,1077501,5000,10.650000,162.869995,10.0,24000.0,2011-12-01,27.65,0,735,...,0,0,0,0,0,0,0,0,0,0
1,1077175,2400,15.960000,84.330002,10.0,12252.0,2011-12-01,8.72,0,735,...,0,0,0,0,0,0,0,1,0,0
2,1076863,10000,13.490000,339.309998,10.0,49200.0,2011-12-01,20.00,0,690,...,0,0,0,0,0,1,0,0,0,0
3,1075269,5000,7.900000,156.460007,3.0,36000.0,2011-12-01,11.20,0,730,...,0,0,0,0,0,0,0,0,0,1
4,1072053,3000,18.639999,109.430000,9.0,48000.0,2011-12-01,5.35,0,660,...,0,0,0,0,0,0,0,0,0,0


In [9]:
x = training_loans